<a href="https://colab.research.google.com/github/9-coding/PyTorch/blob/main/checkpoint_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Checkpoint
모델의 학습 과정 중 특정 시점에서 모델의 가중치와 파라미터를 저장하는 것.
- 중단된 학습 재개
- 과적합 방지 위한 조기 중지

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import os

## Model Implementation

In [ ]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(10, 20)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
# 데이터셋 생성
x_train = torch.randn(100, 10)
y_train = torch.randn(100, 1)
dataset = TensorDataset(x_train, y_train)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 모델, 손실 함수 및 옵티마이저 초기화
model = SimpleModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 체크포인트 디렉토리 생성
checkpoint_dir = './checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

# 최상의 손실 값 초기화
best_loss = float('inf')

## Training Loop
손실이 낮아질 때마다 모델 저장<br>
`torch.save(opject, path)` 사용해 training loop에서 모델의 상태를 저장

In [ ]:
# 훈련 루프
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

    # 손실이 낮아질 때마다 모델 저장
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        checkpoint_path = os.path.join(checkpoint_dir, 'best_model.pth')
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': best_loss,
        }, checkpoint_path)
        print(f'Model saved to {checkpoint_path}')

모델 학습이 끝난 후,<br>
- `torch.load()`를 사용해 모델을 통째로 불러오고
- `torch.nn.Module.load_state_dict(dict)`를 사용해 모델 객체 내의 parameter 값을 초기화함.
- model과 optimizer 둘 다에 대해 초기화하는 것에 유의.

In [ ]:
# 저장된 모델 로드
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

start_epoch = checkpoint['epoch']
best_loss = checkpoint['loss']

print(f'Checkpoint loaded. Training will resume from epoch {start_epoch} with loss {best_loss:.4f}')
